In [ ]:
#Sri Ganesh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')



In [ ]:
df = pd.read_csv('https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/000/940/original/netflix.csv')
df.head()

# ***Understand The Data***

In [ ]:
df.info()

In [ ]:
unique_count = df.nunique()
print(unique_count)
(unique_count/df.shape[0]*100).plot(kind='bar')

In [ ]:
repeted_columns = ['type', 'director', 'cast', 'country', 'date_added','release_year', 'rating', 'duration', 'listed_in']
def plot_value_counts(i,repeted_column):
    a = i.value_counts()
    print(a)
    # fig = px.histogram(a,title=repeted_column,histnorm='percent')
    # fig.show()
    print('*'*100)
for i in repeted_columns:
  plot_value_counts(df[i],i)

# **Analysis Of Missing Data**

1)Is data missing randomly or with a specific pattren

In [ ]:
#Data Columns With Missing Data
df.isna().sum()/df.shape[0]*100

--> How My Data Is Missing

In [ ]:
sns.heatmap(df[df['type']=='Movie'].isna(),cbar=False)

less missing values in movies

In [ ]:
sns.heatmap(df[df['type']=='TV Show'].isna(),cbar=False)

lot of missing values in series than movies

In [ ]:
loc = 1
fig = plt.figure(figsize=(15,10))
for i in ['country','director','cast','date_added']:
  plt.subplot(3,2,loc)
  sns.heatmap(df.sort_values(i).isna(),cbar=False)
  plt.title(i)
  loc+=1

plt.show()



1.   The Data Is Missing In Random
2.   Lot Of missing data in Director Column



In [ ]:
#duration--
df.loc[df['duration'].isna()] #3 Nulls But Mis-Enterd in rating

In [ ]:
df['duration1'] = df['duration']

In [ ]:
df.loc[df['duration'].isna(),'duration'] = df.loc[df['duration'].isna(),'rating'] #copy the info

In [ ]:
#Replace MisEnterd Rating to None
df.loc[df['duration1'].isna(),'rating'] = None

In [ ]:
df['duration'].isna().sum()

In [ ]:
#rating
df['rating'].isna().sum()

In [ ]:
df[df['rating'].isna()].head()

In [ ]:
#Understanding How listed_in is Related to rating
df.groupby('listed_in')['rating'].sum().head()

In [ ]:
#date_added
df['date_added'].value_counts().head()

In [ ]:
#Convert Date TO DateTime
df['date_added'] = pd.to_datetime(df['date_added'].str.strip(),errors='coerce')


In [ ]:
df['date_added'].isna().sum()

We Can Ignore The Nulls has  They are small in number

In [ ]:
#convert relese year to int
df['release_year'] = df['release_year'].astype(int)

In [ ]:
#add month,day,year and weekday columns
df['month'] = df['date_added'].dt.month
df['month_name'] = df['date_added'].dt.month_name()
df['year'] = df['date_added'].dt.year
df['day'] = df['date_added'].dt.day_name()
df['week'] = df['date_added'].dt.weekday

In [ ]:
df['delay'] = df['year']-df['release_year']
df['delay'].value_counts()

In [ ]:
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
} #from chat-gpt tv ratings converted
df['target'] = df['rating'].replace(ratings_ages)

In [ ]:
#fill all the null values
# country  --> we can use the mode
# director --> we cant impute so fill as no Value
# cast --> we cant impute so fill as no Value
df['country']=df['country'].fillna(df['country'].mode()[0])
df['director']=df['director'].fillna('No Value')
df['cast']=df['cast'].fillna('No Value')

# Understanding Nested *Columns*

In [ ]:
#country
country = df['country'].str.strip().str.split(',').explode()
country = country.str.strip()
country.head()

In [ ]:
country_df=df.assign(country=df['country'].str.strip().str.split(',')).explode('country').reset_index()
country_df['country'] = country_df['country'].str.strip()



In [ ]:
country_df[country_df.duplicated()]

In [ ]:
plot_value_counts(country,'country')

In [ ]:
#listed_in

listed_df=df.assign(listed_in=df['listed_in'].str.strip().str.split(',')).explode('listed_in').reset_index()
listed_df['listed_in'] = listed_df['listed_in'].str.strip()



In [ ]:
#director
df['director'].value_counts()
df['count_dir']=df['director'].str.split(',').apply(lambda x:len(x))
px.histogram(df['count_dir'])



we can observe lot of values so needs unnesting

In [ ]:
#director unnest
director_df=df[['director','show_id']].assign(director=df['director'].str.strip().str.split(',')).explode('director').reset_index()
director_df['director'] = director_df['director'].str.strip()

director_df.head()

In [ ]:
#Cast
cast_df=df[['cast','show_id']].assign(cast=df['cast'].str.strip().str.split(',')).explode('cast').reset_index()
cast_df['cast'] = cast_df['cast'].str.strip()
cast_df.head()

# ***Divide The Date into movies and series***

In [ ]:
movies = df[df['type']=='Movie']
series = df[df['type']=='TV Show']

['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description']

In [ ]:
df['type'].value_counts()/df.shape[0]*100

70% movies and 30% series



# ***Analysis ***

In [ ]:
#top 10 Country's:
top_10_country = country.value_counts()[:10]
px.bar(top_10_country,text_auto=True)

In [ ]:
#top 10 Country's:
top_10_bottom_country = country.value_counts()[-10:]
fig = px.bar(top_10_bottom_country,text_auto=True)
fig.show()

In [ ]:
#How Top_10 is Related TO Last 20 to 40 Years
top_10_country_df = country_df[country_df['country'].isin(top_10_country.index)]
fig = px.histogram(top_10_country_df.sort_values('date_added'),x='country',color='type',animation_frame=top_10_country_df['date_added'].dt.year,text_auto=True)
fig.show()

In [ ]:
title = "Percentage OF target audience In Top 10 Countrys"
a = top_10_country_df.groupby(['country','target'])['show_id'].count().reset_index()
a['sum'] = a.groupby(['country'])['show_id'].transform('sum')
a['per']=a['show_id']/a['sum']
px.bar(a.sort_values(['target','per']),y='country',x='per',color='target',text_auto='.0%',title=title)

*  india has lot of teen shows
* spain focus on adult shows

In [ ]:
title = "Percentage OF Movies/Series In Top 10 Countrys"
a = top_10_country_df.groupby(['country','type'])['show_id'].count().reset_index()
a['sum'] = a.groupby(['country'])['show_id'].transform('sum')
a['per']=a['show_id']/a['sum']
px.bar(a.sort_values(['type','per']),y='country',x='per',color='type',text_auto='.0%',title=title)



*   we can see different countries have different distribution
*   south korea has more series than movies
*  in india Netflix has to focus on releasing tv shows




In [ ]:
def abc(x):
  return x.drop_duplicates(['show_id'])['delay'].mean()
title = "Avg Movies Delay in Each Country From Relese to Upload"
a = top_10_country_df[top_10_country_df['type']=='Movie'].groupby(['country'],as_index=False).apply(abc)
a.columns = ['country','delay']
px.bar(a.sort_values('delay'),y='country',x='delay',text_auto='.0',title=title)

India and Germany has the 7 years gap in movie relese and upload

In [ ]:
title = "Avg Series Delay in Each Country From Relese to Upload"
a = top_10_country_df[top_10_country_df['type']=='TV Show'].groupby('country',as_index=False).apply(abc)
a.columns = ['country','delay']
px.bar(a.sort_values('delay'),y='country',x='delay',text_auto='.0',title=title)

*  In Series release and Upload gap is low compared to movies
*  In Japan the delay is high

In [ ]:
exp = '% of the movies are relesed in top_10 Countryes {}'.format(top_10_country.index)
print(top_10_country_df.shape[0]/df.shape[0]*100,exp)

In [ ]:
count_country= country.value_counts()
print("In {} number of movies released are {} which is {}%".format(count_country.index[0],count_country.iloc[0],count_country.iloc[0]/df.shape[0]*100))

In [ ]:
per_country = count_country/df.shape[0]*100
print("Top 2 Countries relese {}% of Movies".format(per_country[:2].sum()))

In [ ]:
title = "Average Number OF movies Relesed in a year in top_10 countryes"
num_mov = top_10_country_df.groupby(['country','year'])['month'].count().reset_index()

In [ ]:
px.bar(num_mov.groupby('country')['month'].mean().round(0).sort_values(ascending=False),text_auto=True,title=title)

In [ ]:
title = "Top 5 Countrys Number Of Movies over Time"
fig = px.line(num_mov[num_mov['country'].isin(count_country[:5].index)],x='year',y='month',color='country',markers=True,title=title)

fig.show()



*   We Can Cearly see that in pandamin the number goes down (trend)

*   Peak is in in 2019



In [ ]:
num_week = top_10_country_df.groupby(['country','week'])['month'].count().reset_index()

In [ ]:
title = "Top 5 Countrys Number Of Movies relesed on weekday"
px.line(num_week[num_week['country'].isin(count_country[:5].index)],x='week',y='month',color='country',markers=True,title=title)



*   we can observe sudden spike in number of movies released on Friday





In [ ]:
#Lets Understand How ratio of movies and series changes over time
type_group = df.groupby(['year','type'])['month'].count().reset_index()


In [ ]:
title = "NUmber Of Movies and Series produced every year Year"
px.line(type_group,x='year',y='month',color='type',title=title)

number movies and series are increasing over time

In [ ]:
type_group['cumsum'] = type_group.groupby('type')['month'].cumsum()
title = "CumSum Of Movies and Series produced every year Year"
px.line(type_group,x='year',y='cumsum',color='type',title=title)

Movies Are Increasing Rapidly Than Series

In [ ]:
type_group['perce']=type_group.groupby('year')['month'].transform(sum)
type_group['perce']=(type_group['month']/type_group['perce'])
title = "Percentage Of Movies and Series produced every year Year"
px.bar(type_group.sort_values(['type','perce']),x='year',y='perce',color='type',title=title,text_auto='.0%')


all the years we have series less than 40% netflix uploading more movies compared to series

In [ ]:
title="Percentage Of Target audians for each year"
type_group = df.groupby(['year','target'],as_index=True)['month'].count().reset_index()
type_group['perce']=type_group.groupby('year')['month'].transform(sum)
type_group['perce']=(type_group['month']/type_group['perce'])
title = "Percentage Of Movies and Series produced every year Year"
px.bar(type_group.sort_values(['target','perce']),x='year',y='perce',color='target',title=title,text_auto='.0%')

in recent years the distribution remains same

In [ ]:
#What is the best time to launch a TV show?
title="Distribution of number movies and series relesed in each day"
px.histogram(df,x='week',color='type',title=title,text_auto=True)

On Friday We Are Seeing lot Of releases

In [ ]:
title="Distribution of number movies and series relesed in each Month"
px.histogram(df.sort_values('month'),x='month_name',color='type',title=title,text_auto=True)

Netflix Is Uploading more in jan,july,dec

In [ ]:
#Director Mearge
dir_df = director_df.merge(df,on='show_id',how='left')
dir_df.head()

In [ ]:
#Analysis of actors/directors of different types of shows/movies.
count_dir=dir_df['director_x'].value_counts()
px.histogram(count_dir[1:])

Most Of The Directors Directed single Movie or series

In [ ]:
top_10_dir=count_dir[1:11]
top_10_dir.plot(kind='bar')

In [ ]:
top_10_dir_df = dir_df[dir_df['director_x'].isin(top_10_dir.index)]
top_10_dir_df['type'].value_counts()

so Needs A seperate Movies Top 10 And Series TOp 10

In [ ]:
#Movies Top 10 Directors
movies_top_10_dir = dir_df[dir_df['type']=='Movie']['director_x'].value_counts()[1:11]
series_top_10_dir = dir_df[dir_df['type']=='TV Show']['director_x'].value_counts()[1:11]


In [ ]:
movies_top_dir_df = dir_df[dir_df['director_x'].isin(movies_top_10_dir.index)]
series_top_dir_df = dir_df[dir_df['director_x'].isin(series_top_10_dir.index)]

In [ ]:
movies_top_10_dir.plot(kind='bar')

Top Directors

In [ ]:
series_top_10_dir.plot(kind='bar')
print((series['director']=='No Value').sum(),(movies['director']=='No Value').sum())

lot of null values so above may not be true

In [ ]:
#rating and target
title = "number of Movies and Series in rating and target category"
a = df.groupby(['type','target'],as_index=False)['rating'].count()
a['sum'] = a.groupby(['target'])['rating'].transform('sum')
a['per']=(a['rating']/a['sum'])
px.bar(a.sort_values(['type','per']),x='per',y='target',color='type',text_auto='.0%',title=title)

*  We Have more adult and teen movies and series
* In Kids category series are more common than movies

In [ ]:
count_genre = listed_df['listed_in'].value_counts()
count_genre.head()

In [ ]:
px.bar(count_genre[:10],text_auto=True)

In [ ]:
title = "Famous Genre in Each top 10 Countr's For Movies"
a = listed_df[(listed_df['type']=='Movie') & (listed_df['country'].isin(top_10_country.index)) ].groupby(['country','listed_in'])['title'].count().reset_index()
px.sunburst(a,path=['country','listed_in'],values='title',title=title)

* International Movies and Drams are popular in India
* Dramas Comedy and Documentary are Popular Genre In US

In [ ]:
title = "Famous Genre in Each top 10 Countr's For Series"
a = listed_df[(listed_df['type']=='TV Show') & (listed_df['country'].isin(top_10_country.index)) ].groupby(['country','listed_in'])['title'].count().reset_index()
px.sunburst(a,path=['country','listed_in'],values='title',title=title)

* TV Comedies and Dramas are Popular in US
* Brtish TV Shows in UK

In [ ]:
title = "Each Year Genre For Movies"
a = listed_df[listed_df['type']=='Movie'].groupby(['year','listed_in'])['title'].count().reset_index()
px.sunburst(a,path=['year','listed_in'],values='title',title=title)

* International Movies
* Drama
* Comedy
* Action and Adventures

Are popular in Movies all Years



In [ ]:
title = "Each Year Genre For Series"
a = listed_df[listed_df['type']=='TV Show'].groupby(['year','listed_in'])['title'].count().reset_index()
px.sunburst(a,path=['year','listed_in'],values='title',title=title)

* International TV Shows
* TV Dramas
* TV comedys
* TV Kinds
Are Popouler in Series All Years

In [ ]:
title = "Number Of Movies in Each Genre"
movies_genre = df[df['type']=='Movie']['listed_in'].str.replace(', ',',').replace(' ,',',').str.split(',').explode()
top_movies_genre = movies_genre.value_counts()[:10]
px.bar(top_movies_genre,text_auto=True,title=title)

In [ ]:
title = "Number Of Series in each genre"
series_genre = df[df['type']=='TV Show']['listed_in'].str.replace(', ',',').replace(' ,',',').str.split(',').explode()
top_series_genre = series_genre.value_counts()[:10]
px.bar(top_series_genre,text_auto=True,title=title)

In [ ]:
#duration
title = "Movie Duration distribution"
movies['duration1'] = movies['duration'].str.split(' ').apply(lambda x:int(x[0]))
px.histogram(movies['duration1'],text_auto=True,title=title)

More Movies Are more Than around 100 min

In [ ]:
title = "Seasons distribution"
px.histogram(series['duration'],text_auto=True,title=title)

Most number of series are 1,2,3,4 seasons long

In [ ]:
#Cast
cast_list = df[df['type']=='Movie']['cast'].dropna().str.split(',').sum()
top_cast = pd.Series(cast_list).value_counts()[1:11]
title = "Top Cast in Movies"
px.bar(top_cast,title = title,text_auto=True)

In [ ]:
#Cast
cast_list = df[df['type']=='TV Show']['cast'].dropna().str.split(',').sum()
top_cast = pd.Series(cast_list).value_counts()[1:11]
title = "Top Cast in Series"
px.bar(top_cast,title = title,text_auto=True)

# **Insights:-**
1. Netflix has more movies (70%) than TV shows (30%), suggesting a preference for single-consumption content.
2. Content additions peaked in 2020, likely due to the COVID-19 pandemic.
3. The USA contributes the most content (60%), showing a Western focus.
4. 'Drama,'Comedy','international TV Showa' are the most common genres, catering to broad tastes.
5. "TV-MA" rated content dominates, indicating a strong adult audience base.
6. Movies average 90-120 minutes, while most TV shows have 1-2 seasons.
7. South Korea excels in "Thriller" and "Romance" driven by the global popularity of K-dramas.
8. Recent additions include more family-friendly titles, hinting at a strategy shift.
9. Most content originates from the 2010s

# **Recommendations:-**
1. Target Emerging Markets: Invest in countries like South Korea and India.
2. Expand TV Shows: Develop more episodic content for better viewer retention.
3. Diversify Genres: Focus on underrepresented genres to attract niche audiences.
4. Family Content: Add more kid-friendly and family-oriented shows

In [ ]:
print("Thank You")

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________